In [6]:
#. 라이브러리 import
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from fancyimpute import KNN, SoftImpute, BiScaler

In [5]:
# excel(xls) 데이터 파일 불러오기
dataset = pd.read_excel('step_hardness_edited.xls')
dataset_arr = np.array(dataset)
dataset.head(20)

,PRODUCT,SIZE,MATERIAL,THICKNES,WIDTH,SPECIES,HRNO,SP_NO,ST_NO,YP,...,SI,MN,P,S,TAL,SOL,N,DAYOUT,D_CODE,_OBSTAT_
0,A26930,JS-MRT4CA,BRC4,0.20,895,W06D2,J063710,Y40256,111,31.2,...,5,30,14,12,36,33,23,980914,1,01101 0 0 0
1,A26931,JS-MRT4CA,BRC4,0.20,895,W06D2,J089330,Y40884,211,30.2,...,13,31,12,9,40,37,30,980922,1,01101 0 0 0
2,A27034,JS-MRT4CA,BRC4,0.21,878,W06D2,J095750,Y41031,711,32.0,...,5,28,13,10,39,36,38,980925,1,01101 0 0 0
3,A27033,JS-MRT4CA,BRC4,0.21,878,W06D2,J095740,Y41031,811,33.9,...,5,28,13,10,39,36,38,980925,1,01101 0 0 0
4,A27179,JS-MRT4CA,BRC4,0.23,872,W06D2,J095820,Y41033,111,30.0,...,5,29,11,10,40,37,44,980925,1,01101 0 0 0
5,A27037,JS-MRT4CA,BRC4,0.24,864,W06D2,J095840,Y41033,511,30.8,...,5,29,11,10,40,37,44,980925,1,01101 0 0 0
6,A27178,JS-MRT4CA,BRC4,0.23,872,W06D2,J095810,Y41033,211,31.9,...,5,29,11,10,40,37,44,980925,1,01101 0 0 0
7,A27071,JS-MRT4CA,BRC4,0.28,890,W06D2,J098730,Y41031,511,30.0,...,5,28,13,10,39,36,38,980925,1,01101 0 0 0
8,A27180,JS-MRT4CA,BRC4,0.24,864,W06D2,J098760,Y41033,411,29.1,...,5,29,11,10,40,37,44,980925,1,01101 0 0 0
9,A27177,JS-MRT4CA,BRC4,0.23,872,W06D2,J098770,Y41033,311,34.0,...,5,29,11,10,40,37,44,980925,1,01101 0 0 0


In [ ]:
X = dataset_arr[:,11:27].astype(float)
X.shape

In [14]:
# 학습/검증 데이터 나누기
# idx 생성
idx_row = np.arange(39)
idx_col = np.arange(16)

from itertools import product

non_missing_idx = []
for row, col in product(idx_row, idx_col):
    non_missing_idx.append([row, col])

non_missing_idx = np.array(non_missing_idx)
print(non_missing_idx)

[[ 0  0]
 [ 0  1]
 [ 0  2]
 ...
 [38 13]
 [38 14]
 [38 15]]


In [18]:
idx = np.arange(non_missing_idx.shape[0])

train_idx, test_idx = train_test_split(idx, test_size=0.1)

X_incomplete = X.copy()
X_incomplete[non_missing_idx[test_idx, 0], non_missing_idx[test_idx, 1]] = np.nan

print('Original')
print(pd.DataFrame(X[:, :5]))

print('')

print('Incomplete')
print(pd.DataFrame(X_incomplete[:, :5]))

Original
       0      1     2     3      4
0   31.0  177.0  60.0  0.48  384.0
1   29.0  186.0  61.8  0.38  385.0
2   28.0  180.0  60.4  0.45  386.0
3   27.0  178.0  61.0  0.44  385.0
4   31.0  180.0  59.3  0.42  400.0
5   33.0  188.0  59.0  0.48  394.0
6   31.0  184.0  58.4  0.44  400.0
7   33.0  170.0  58.0  0.36  330.0
8   32.0  190.0  58.2  0.48  380.0
9   28.0  180.0  59.2  0.44  395.0
10  30.0  190.0  60.0  0.32  383.0
11  30.0  187.0  60.2  0.43  371.0
12  29.0  172.0  60.6  0.46  380.0
13  30.0  177.0  60.4  0.47  299.0
14  31.0  171.0  61.0  0.42  354.0
15  33.0  190.0  59.0  0.46  380.0
16  32.0  176.0  60.6  0.48  375.0
17  31.0  176.0  60.8  0.42  380.0
18  22.0  180.0  59.4  0.40  379.0
19  33.0  179.0  59.4  0.37  381.0
20  27.0  173.0  63.2  0.46  335.0
21  28.0  175.0  60.4  0.41  380.0
22  31.0  180.0  59.0  0.38  355.0
23  30.0  179.0  60.2  0.54  317.0
24  29.0  177.0  59.4  0.35  261.0
25  29.0  163.0  59.8  0.32  311.0
26  28.0  163.0  59.9  0.33  326.0
27  30.0  1

In [5]:
# Simple imputatiion
imputer_mean = SimpleImputer(strategy='mean')
imputer_median = SimpleImputer(strategy='median')

imputer_mean.fit(X_incomplete)
imputer_median.fit(X_incomplete)

X_filled_mean = imputer_mean.transform(X_incomplete)
X_filled_median = imputer_median.transform(X_incomplete)

In [6]:
# True value at missing position
missing_true = X[non_missing_idx[test_idx, 0], non_missing_idx[test_idx, 1]]

# Prediction value at missing position (mean / median)
missing_pred_mean = X_filled_mean[non_missing_idx[test_idx, 0], non_missing_idx[test_idx, 1]]
missing_pred_median = X_filled_median[non_missing_idx[test_idx, 0], non_missing_idx[test_idx, 1]]

# MSE
mse_mean = mean_squared_error(missing_true, missing_pred_mean)
mse_median = mean_squared_error(missing_true, missing_pred_median)

print("MSE-Mean : %f" % mse_mean)
print("MSE-Median : %f" % mse_median)

MSE-Mean : 427.170344
MSE-Median : 613.149494


In [7]:
# KNN imputation (k=5)
imputer_knn = 
imputer_knn.fit(X_incomplete)

X_filled_knn = 

missing_pred_knn = 

mse_knn = 
print("MSE-KNN : %f" % mse_knn)

Imputing row 1/39 with 2 missing, elapsed time: 0.002
MSE-KNN : 125.931874


In [8]:
# biscaler
biscaler = BiScaler()
biscaler.fit(X_incomplete)

X_incomplete_normalized = 

[BiScaler] Initial log residual value = 15.110003
[BiScaler] Iter 1: log residual = 1.105843, log improvement ratio=14.004160
[BiScaler] Iter 2: log residual = -0.234926, log improvement ratio=1.340769
[BiScaler] Iter 3: log residual = -0.937462, log improvement ratio=0.702535
[BiScaler] Iter 4: log residual = -1.772534, log improvement ratio=0.835072
[BiScaler] Iter 5: log residual = -2.718688, log improvement ratio=0.946154
[BiScaler] Iter 6: log residual = -3.722262, log improvement ratio=1.003574
[BiScaler] Iter 7: log residual = -4.741860, log improvement ratio=1.019598
[BiScaler] Iter 8: log residual = -5.758027, log improvement ratio=1.016167
[BiScaler] Iter 9: log residual = -6.765381, log improvement ratio=1.007354
[BiScaler] Iter 10: log residual = -7.764451, log improvement ratio=0.999070
[BiScaler] Iter 11: log residual = -8.757268, log improvement ratio=0.992817
[BiScaler] Iter 12: log residual = -9.745746, log improvement ratio=0.988477
[BiScaler] Iter 13: log residual = 

In [9]:
imputer_knn2 = 
imputer_knn2.fit(X_incomplete_normalized)

X_filled_knn_normalized = 
X_filled_knn2 = 

missing_pred_knn2 = 

mse_knn2 = 
print("MSE-KNN-Biscaler : %f" % mse_knn2)

Imputing row 1/39 with 2 missing, elapsed time: 0.002
MSE-KNN-Biscaler : 61.828935


In [10]:
# Soft-impute
imputer_soft = SoftImpute()
imputer_soft.fit(X_incomplete)

X_filled_soft = 

[SoftImpute] Max Singular Value of X_init = 7323.440496
[SoftImpute] Iter 1: observed MAE=10.314676 rank=8
[SoftImpute] Iter 2: observed MAE=10.217949 rank=8
[SoftImpute] Iter 3: observed MAE=10.111472 rank=7
[SoftImpute] Iter 4: observed MAE=10.057916 rank=6
[SoftImpute] Iter 5: observed MAE=9.830119 rank=6
[SoftImpute] Iter 6: observed MAE=9.600833 rank=6
[SoftImpute] Iter 7: observed MAE=9.391847 rank=6
[SoftImpute] Iter 8: observed MAE=9.332324 rank=6
[SoftImpute] Iter 9: observed MAE=9.343685 rank=5
[SoftImpute] Iter 10: observed MAE=9.001171 rank=5
[SoftImpute] Iter 11: observed MAE=8.702698 rank=5
[SoftImpute] Iter 12: observed MAE=8.344832 rank=4
[SoftImpute] Iter 13: observed MAE=8.171056 rank=3
[SoftImpute] Iter 14: observed MAE=7.866246 rank=3
[SoftImpute] Iter 15: observed MAE=7.558729 rank=2
[SoftImpute] Iter 16: observed MAE=7.362870 rank=2
[SoftImpute] Iter 17: observed MAE=7.295671 rank=2
[SoftImpute] Iter 18: observed MAE=7.271003 rank=2
[SoftImpute] Iter 19: observed 

In [11]:
missing_pred_soft = 

mse_soft = 
print("MSE-Soft : %f" % mse_soft)

MSE-Soft : 416.801524


In [12]:
imputer_soft2 = 
imputer_soft2.fit(X_incomplete_normalized)

X_filled_soft_normalized = 
X_filled_soft2 = 

[SoftImpute] Max Singular Value of X_init = 12.833248
[SoftImpute] Iter 1: observed MAE=0.030971 rank=15
[SoftImpute] Iter 2: observed MAE=0.031216 rank=15
[SoftImpute] Iter 3: observed MAE=0.031496 rank=15
[SoftImpute] Iter 4: observed MAE=0.031801 rank=15
[SoftImpute] Iter 5: observed MAE=0.032117 rank=15
[SoftImpute] Iter 6: observed MAE=0.032402 rank=15
[SoftImpute] Iter 7: observed MAE=0.032674 rank=15
[SoftImpute] Iter 8: observed MAE=0.032843 rank=16
[SoftImpute] Iter 9: observed MAE=0.032914 rank=16
[SoftImpute] Iter 10: observed MAE=0.032981 rank=16
[SoftImpute] Iter 11: observed MAE=0.033048 rank=16
[SoftImpute] Iter 12: observed MAE=0.033113 rank=16
[SoftImpute] Iter 13: observed MAE=0.033174 rank=16
[SoftImpute] Iter 14: observed MAE=0.033229 rank=16
[SoftImpute] Iter 15: observed MAE=0.033283 rank=16
[SoftImpute] Iter 16: observed MAE=0.033332 rank=16
[SoftImpute] Iter 17: observed MAE=0.033377 rank=16
[SoftImpute] Iter 18: observed MAE=0.033418 rank=16
[SoftImpute] Iter 1

In [13]:
missing_pred_soft2 = 

mse_soft2 = 
print("MSE-Soft-Biscaler : %f" % mse_soft2)

MSE-Soft-Biscaler : 65.829501
